# View Downtime por Causa

Downtime agregado por tipo de manutenção e equipamento.

In [ ]:
# Configuração (lê parâmetros do job)
try:
    CATALOG = dbutils.widgets.get('catalog')
except:
    CATALOG = 'manufatura_lakehouse'

try:
    SCHEMA_SILVER = dbutils.widgets.get('schema_silver')
except:
    SCHEMA_SILVER = 'silver'

try:
    SCHEMA_GOLD = dbutils.widgets.get('schema_gold')
except:
    SCHEMA_GOLD = 'gold'

def fqtn(schema, table):
    if CATALOG and CATALOG.lower() != 'none':
        return f"`{CATALOG}`.`{schema}`.`{table}`"
    else:
        return f"`{schema}`.`{table}`"

# Criar schema se necessário
if CATALOG and CATALOG.lower() != 'none':
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{CATALOG}`.`{SCHEMA_GOLD}`")
    spark.sql(f"USE CATALOG `{CATALOG}`")
else:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{SCHEMA_GOLD}`")


In [ ]:
spark.sql(f"""
CREATE OR REPLACE VIEW {fqtn(SCHEMA_GOLD, 'vw_downtime_por_causa')} AS
SELECT 
    t.full_date,
    e.equipment_name,
    tm.maintenance_type,
    sum(f.downtime_minutes) as total_downtime_minutes,
    count(*) as maintenance_count,
    avg(f.downtime_minutes) as avg_downtime_minutes
FROM {fqtn(SCHEMA_GOLD, 'fact_manutencao')} f
JOIN {fqtn(SCHEMA_GOLD, 'dim_tempo')} t ON f.start_date_key = t.date_key
JOIN {fqtn(SCHEMA_GOLD, 'dim_equipamento')} e ON f.equipment_id = e.equipment_id
LEFT JOIN {fqtn(SCHEMA_GOLD, 'dim_tipo_manutencao')} tm ON f.maintenance_type = tm.maintenance_type
GROUP BY 1, 2, 3
ORDER BY total_downtime_minutes DESC
""")